In [1]:
%matplotlib inline
from ipywidgets import interactive
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import nltk
import string
import re

In [2]:
train = pd.read_csv('balanced_2classes_train.csv', encoding='utf-8')
train.head()

,label,user_id,book_id,text
0,0,4969402,10706934,لا جديد بالنسبة لي!
1,0,4865181,3153116,دعونا لا ننكر أن لغة علوان هنا ، جمـيلة جداً ...
2,0,8132581,3831344,الكتاب كان دون توقعاتي لعله بسبب كرهي للفلسفة...
3,0,8553746,11205152,مشدنيش في حاجة . وسلبي بشكل كبير ويميل للكآبه...
4,0,5115054,11942408,شغاال برضه بس مش واو!!


In [3]:
train['text'][1]

' دعونا لا ننكر أن لغة علوان هنا ، جمـيلة جداً لا يخفى عليكم أني انبهرت بالكم الهائل من الكنايات والتعابير والمفردات الرائعة لكن سرعان ما انتبهت لسؤال ظل يراودني خلال الصفحات : أهي حقاً رواية ؟ لم أستشعر ذلك حقاً ، الحبكة ضعيفة نوعاً ما كما أن الفكرة ليست ما أحبذه ، الحب الحب الحب . ثم إلى أين ؟'

In [4]:
val = pd.read_csv('balanced_2classes_val.csv', encoding='utf-8')
val.head()

,label,user_id,book_id,text
0,1,12977458,6016845,أماية وانت بترحي الرحى على مفارق ضحى وحدك وبت...
1,1,2001255,5899456,شعر شديد التميز خاصتا الأشعار الوطنيه من ينسى...
2,0,2698419,10664655,الكتاب يتلخّص في: الليبرالية العربية = المرأه...
3,1,13320549,7709594,تتحدث عن ثنائية الدين و العمل و ثنائية العدل ...
4,1,2097619,2073853,ديوانى المفضل


In [5]:
test = pd.read_csv('balanced_2classes_test.csv', encoding='utf-8')
test.head()

,label,user_id,book_id,text
0,0,5541253,3503947,طاب حد يقنعني!! حاسس إني ضيعت وقتي
1,0,4077563,3031309,"سهل اللغة وبسيط الأسلوب. أعتبره كـ ""وجبة خفيف..."
2,0,6348545,10098640,حسيت انها اعادة لفكرة الكتاب الاول بفضفضة اكت...
3,0,2106483,10706934,الكتاب للمفرطين في التفاؤل، ولمن يريدون تضعية...
4,0,9533334,16081961,مملة! لم تجذبني الا في مواضع معدودة، قد يكون ...


In [6]:
stops = pd.read_excel('ar_stops.xlsx', encoding='utf-8')
stops.head()

,“
0,[
1,[
2,وأنه
3,وأني
4,“


In [7]:
print(train.shape)
print(val.shape)
print(test.shape)

(9872, 4)
(3288, 4)
(3288, 4)


In [8]:
frames = [train, val, test]

In [9]:
data = pd.concat(frames)

In [10]:
data.head()

,label,user_id,book_id,text
0,0,4969402,10706934,لا جديد بالنسبة لي!
1,0,4865181,3153116,دعونا لا ننكر أن لغة علوان هنا ، جمـيلة جداً ...
2,0,8132581,3831344,الكتاب كان دون توقعاتي لعله بسبب كرهي للفلسفة...
3,0,8553746,11205152,مشدنيش في حاجة . وسلبي بشكل كبير ويميل للكآبه...
4,0,5115054,11942408,شغاال برضه بس مش واو!!


In [11]:
print(data.shape)
print(train.shape[0] + val.shape[0] + test.shape[0])

(16448, 4)
16448


In [12]:
data['text'] = data['text'].apply(lambda x: re.sub(r"[0-9]", " ", x))
#data['text'] = data['text'].apply(lambda x: re.sub(r"[]", " ", x))

In [13]:
# stop words and punct
stop_words = list(stops['“'])

In [14]:
stop_words

['[',
 '[',
 'وأنه',
 'وأني',
 '“',
 '”',
 'الله',
 'عددا',
 'بها',
 'ولكن',
 'أنه',
 'فهذا',
 'بأن',
 'فهذا',
 'لو',
 'فيه',
 'يا',
 'إذا',
 'و',
 'منها',
 'فيه',
 'ذو',
 'مما',
 '؛',
 'مع',
 'وحين',
 'ونحو',
 'بما',
 'ومع',
 'وبهم',
 'بنا',
 'فأي',
 'أو',
 'به',
 'بها',
 'ان',
 'ولكن',
 'بأن',
 'عليك',
 'إذا',
 'فإنه',
 'يا',
 'بعض',
 'ذلك',
 'لأن',
 'وإن',
 'فيه',
 'ان',
 'أما',
 'وأما',
 'كما',
 'بما',
 'عليها',
 'له',
 'لمن',
 'وقد',
 'وهو',
 'فهنا',
 'وكذلك',
 'وإذا',
 'أيضاً',
 'عليك',
 'وأنه',
 'أيضاً',
 'ومن',
 'لأنها',
 'إليه',
 'بن',
 'أي',
 'بعض',
 'عند',
 'عليه',
 'بل',
 'أبو',
 'لهم',
 'هذا',
 'فهذا',
 'بها',
 'فإنه',
 'بذلك',
 'وأما',
 'وإن',
 'وقد',
 'وهو',
 'هذا',
 'بأن',
 'وإن',
 'بعضكم',
 'وكذلك',
 'فقد',
 'لأنها',
 'وحين',
 'ولكن',
 '·',
 'إنما',
 'عليهم',
 'غيرها',
 'فيما',
 'قال',
 'يقول',
 'قلت',
 'بقوله',
 'يقال',
 'فقال',
 'يقال',
 'فقالت',
 'كونه',
 'بعضها',
 'بهذه',
 'فلما',
 'قيل',
 'وعند',
 'بهما',
 'حينئذ',
 'ذوو',
 'فقيل',
 'وبه',
 'وكذا',
 'ولهذا',
 'الك

In [200]:
# def remove_stops(words):
#     return {
#         word for word in words \
#         if not word in useless_words}

In [201]:
review = data['text']
review.head()

0                                  لا جديد بالنسبة لي!
1     دعونا لا ننكر أن لغة علوان هنا ، جمـيلة جداً ...
2     الكتاب كان دون توقعاتي لعله بسبب كرهي للفلسفة...
3     مشدنيش في حاجة . وسلبي بشكل كبير ويميل للكآبه...
4                               شغاال برضه بس مش واو!!
Name: text, dtype: object

In [202]:
# word tokenization
nltk.download('punkt')
from nltk.tokenize import word_tokenize
review = review.apply(lambda x: word_tokenize(x))

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\CompuTek\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [203]:
review.head()

0                           [لا, جديد, بالنسبة, لي, !]
1    [دعونا, لا, ننكر, أن, لغة, علوان, هنا, ،, جمـي...
2    [الكتاب, كان, دون, توقعاتي, لعله, بسبب, كرهي, ...
3    [مشدنيش, في, حاجة, ., وسلبي, بشكل, كبير, ويميل...
4                     [شغاال, برضه, بس, مش, واو, !, !]
Name: text, dtype: object

In [204]:
review = review.apply(lambda words: [w for w in words if w not in stop_words])

In [205]:
review.head()

0                                  [لا, جديد, بالنسبة]
1    [دعونا, لا, ننكر, لغة, علوان, جمـيلة, جداً, لا...
2    [الكتاب, توقعاتي, كرهي, للفلسفة, ولعله, أسلوب,...
3     [مشدنيش, حاجة, وسلبي, كبير, ويميل, للكآبه, اكبر]
4                                [شغاال, برضه, بس, مش]
Name: text, dtype: object

In [206]:
from nltk.stem.isri import ISRIStemmer

In [207]:
review = review.apply(lambda words: [ISRIStemmer().stem(w) for w in words])

In [208]:
review

0                                          [لا, جدد, نسب]
1       [دعو, لا, نكر, لغة, علو, جمـيل, جدا, لا, خفى, ...
2       [كتب, وقع, كره, لسف, لعل, سلب, د.المسيري, تقعر...
3                  [مشدنيش, حجة, سلب, كبر, يمل, كآب, كبر]
4                                     [شغال, برض, بس, مش]
5                                              [حلى, قصد]
6       [روي, تحت, توقع, كثر, بلغ, عبر, رئع, صمت, حنن,...
7       [كتب, درويش, بلغ, قرب, هذي, قرب, جده, كثر, علي...
8       [كنت, توقع, كتب, عكس, نضج, شخص, حمد, حلم, زي, ...
9       [طول, حيت, لم, ختر, ولم, قرر, انن, قرر, انن, خ...
10      [قرت, عرف, عجب, قصص, حفظ, صغر, ونت, سعد, لبد, ...
11      [بدع, علو, رومنس, لمذ, يتج, غيرها؟؟, رغم, غرق,...
12      [فرق, مصر, ثور, مصر, ثور, قبل, ريه, ووب, بدن, ...
13      [ذكر, أسلوب, مرح, جمل, يعا, امر, سيئ, يرد, حسن...
14      [كتب, رفه, نفس, جعل, اضح, قلب, سحق, ٤, نجم, .ل...
15      [برأ, فرض, سمى, بـ, حلى, قبح, قصد, حقق, شمل, ج...
16      [قيمى, خدش, بيه, عشن, شفت, يلم, اقر, روي, وده,...
17      [كتب, 

In [209]:
review = review.apply(lambda x: " ".join(x))

In [210]:
review.head()

0                                           لا جدد نسب
1    دعو لا نكر لغة علو جمـيل جدا لا خفى علي اني به...
2    كتب وقع كره لسف لعل سلب د.المسيري تقعر عالأقل ...
3                       مشدنيش حجة سلب كبر يمل كآب كبر
4                                       شغال برض بس مش
Name: text, dtype: object

In [212]:
train = data[:train.shape[0]]
val = data[train.shape[0]: (val.shape[0] + train.shape[0])]
test = data[(val.shape[0] + train.shape[0]):]

In [216]:
train.shape[0] + val.shape[0] + test.shape[0]

16448

In [217]:
X_train = train['text']
y_train = train['label']

X_val = val['text']
y_val = val['label']

X_test = test['text']
y_test = test['label']

In [218]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score

In [219]:
train_df_vectorized = TfidfVectorizer(min_df=2, ngram_range=(1, 3))
X = train_df_vectorized.fit_transform(X_train) 

In [220]:
clfrNB = MultinomialNB(alpha = 0.1)
clfrNB.fit(X, y_train)

MultinomialNB(alpha=0.1, class_prior=None, fit_prior=True)

In [221]:
from sklearn.model_selection import cross_val_score
cv = cross_val_score(clfrNB, X, y_train,cv=10)

In [222]:
cv

array([0.81072874, 0.79453441, 0.81053698, 0.80141844, 0.79736575,
       0.79533941, 0.82066869, 0.81357649, 0.81661601, 0.8064843 ])

In [223]:
cv.mean()

0.8067269236922092

In [224]:
test = ['لم احبه']
test = train_df_vectorized.transform(test)
clfrNB.predict(test)

array([0], dtype=int64)

In [225]:
test = ['جميل جدا']
test = train_df_vectorized.transform(test)
clfrNB.predict(test)

array([1], dtype=int64)

In [226]:
test = ['عظيم جدا']
test = train_df_vectorized.transform(test)
clfrNB.predict(test)

array([1], dtype=int64)

In [227]:
X_val

0        أماية وانت بترحي الرحى على مفارق ضحى وحدك وبت...
1        شعر شديد التميز خاصتا الأشعار الوطنيه من ينسى...
2        الكتاب يتلخّص في: الليبرالية العربية = المرأه...
3        تتحدث عن ثنائية الدين و العمل و ثنائية العدل ...
4                                           ديوانى المفضل
5        كتاب ممتع جميل قرأته منذ سنوات و اسعد بالرجوع...
6        تعبر عن واقع فئة من المجتمع . أختلف مع الراوي...
7        لروعتها تجرعتها دفعة واحدة . اكتبي أيتها البث...
8        الكتاب كان متخما بالجمال حتى العصر الأموي. بع...
9        اسلوب ممتع في سرد السيرة الذاتية بطريقة روائي...
10       الكاتب أبدع في هذه الرواية بقدرته على السرد و...
11       أشعر وكأن القصة حُكيت في أذني قبل أن أقرأها. ...
12       جيد، مناسب للشباب. على الأقل من الممكن أن يكو...
13       الرواية مبهره جدا أشعر بأن مجهود الكتابه والب...
14       جررررررررررررررررررئ لابعد حد ايه ده . !!!!!!...
15       فلسفته مثيره , افكاره خطيره اسلوبه ولغته وكل ...
16       مش عارف الحقيقة اذا كنت ظلمت الرواية ولا لأ, ...
17       نزار 

In [228]:
validation = train_df_vectorized.transform(X_val.values) 

In [230]:
predicto = clfrNB.predict(validation)

In [236]:
from sklearn.metrics import accuracy_score
accuracy_score(y_val, predicto)

0.8044403892944039

In [237]:
from sklearn.metrics import confusion_matrix
conf = confusion_matrix(y_val, predicto)
conf

array([[1315,  305],
       [ 338, 1330]], dtype=int64)

In [238]:
from sklearn.metrics import precision_score, recall_score
precision = precision_score(y_val, predicto)
precision

0.8134556574923547

In [239]:
recall = recall_score(y_val, predicto)
recall

0.7973621103117506